<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

## *Data Science Unit 4 Sprint 4*

# Sprint Challenge
### RNNs, CNNs, GANS, and AutoML

In this Sprint Challenge, you'll explore some of the cutting edge of Data Science. *Caution* - these approaches can be pretty heavy computationally. All problems are designed to completed with 5-10 minutes of run time on most machines. If you approach takes longer, please double check your work. 

## Part 1 - RNNs

Use an RNN to fit a classification model on tweets to distinguish from tweets from any two accounts. The following code sample illustrates how to access data from an account (no API auth needed, uses [twitterscraper](https://github.com/taspinar/twitterscraper): 

In [1]:
!pip install twitterscraper

In [5]:
from twitterscraper import query_tweets

austen_tweets = query_tweets('from:austen', 1000)
len(austen_tweets)

INFO: queries: ['from:austen since:2006-03-21 until:2006-11-16', 'from:austen since:2006-11-16 until:2007-07-14', 'from:austen since:2007-07-14 until:2008-03-10', 'from:austen since:2008-03-10 until:2008-11-06', 'from:austen since:2008-11-06 until:2009-07-04', 'from:austen since:2009-07-04 until:2010-03-01', 'from:austen since:2010-03-01 until:2010-10-27', 'from:austen since:2010-10-27 until:2011-06-25', 'from:austen since:2011-06-25 until:2012-02-20', 'from:austen since:2012-02-20 until:2012-10-17', 'from:austen since:2012-10-17 until:2013-06-14', 'from:austen since:2013-06-14 until:2014-02-10', 'from:austen since:2014-02-10 until:2014-10-08', 'from:austen since:2014-10-08 until:2015-06-05', 'from:austen since:2015-06-05 until:2016-01-31', 'from:austen since:2016-01-31 until:2016-09-28', 'from:austen since:2016-09-28 until:2017-05-26', 'from:austen since:2017-05-26 until:2018-01-21', 'from:austen since:2018-01-21 until:2018-09-18', 'from:austen since:2018-09-18 until:2019-05-17']
INFO

181

In [6]:
austen_tweets[0].text

'I love love love working with great people.pic.twitter.com/fCKOm6Vl'

Your Tasks:
* Select two twitter accounts to gather data from
* Use twitterscraper to get ~1,000 tweets from each account
* Encode the characters to a sequence of integers for the model
* Get the data into the appropriate shape/format, including labels and a train/test split
* Use Keras to fit a predictive model, classying tweets as being from one acount or the other
* Report your overall score and accuracy

For reference, the [Keras IMDB classification example](https://github.com/keras-team/keras/blob/master/examples/imdb_lstm.py) will be useful, as well as the RNN code we used in class.

Note - focus on getting a running model, not on making accuracy with extreme data size or epoch numbers. Fit a baseline model based on tweet text. Only revisit and push accuracy or incorporate additional features if you get everything else done!

In [7]:
elon_tweets = query_tweets('from:elonmusk', 1000)
len(elon_tweets)

INFO: queries: ['from:elonmusk since:2006-03-21 until:2006-11-16', 'from:elonmusk since:2006-11-16 until:2007-07-14', 'from:elonmusk since:2007-07-14 until:2008-03-10', 'from:elonmusk since:2008-03-10 until:2008-11-06', 'from:elonmusk since:2008-11-06 until:2009-07-04', 'from:elonmusk since:2009-07-04 until:2010-03-01', 'from:elonmusk since:2010-03-01 until:2010-10-27', 'from:elonmusk since:2010-10-27 until:2011-06-25', 'from:elonmusk since:2011-06-25 until:2012-02-20', 'from:elonmusk since:2012-02-20 until:2012-10-17', 'from:elonmusk since:2012-10-17 until:2013-06-14', 'from:elonmusk since:2013-06-14 until:2014-02-10', 'from:elonmusk since:2014-02-10 until:2014-10-08', 'from:elonmusk since:2014-10-08 until:2015-06-05', 'from:elonmusk since:2015-06-05 until:2016-01-31', 'from:elonmusk since:2016-01-31 until:2016-09-28', 'from:elonmusk since:2016-09-28 until:2017-05-26', 'from:elonmusk since:2017-05-26 until:2018-01-21', 'from:elonmusk since:2018-01-21 until:2018-09-18', 'from:elonmusk 

721

In [11]:
for i, j in enumerate(austen_tweets):
    if i < 10:
        print(austen_tweets[i].text)

print("-"*100)
for i, j in enumerate(elon_tweets):
    if i < 10:
        print(elon_tweets[i].text)

I love love love working with great people.pic.twitter.com/fCKOm6Vl
...not even close
What if you’ve been trying for hundreds of years and it’s never come even remotely close to working but millions of people have died as a result?https://twitter.com/franifio/status/1041514361053822976 …
That plus a little behavioral stuff is where we’re heading
This account is fascinating. Hard to imagine how bad 2008 washttps://twitter.com/TBTFLive/status/1041683159992156161 …
Don’t get your news from memes
So this is a non-standard thing for me to be really happy about, but very smart people being pushed to their limits with a support system and community to fall back on in the hard times is what makes Lambda School great.

“If I were on my own I would have failed by now.”https://twitter.com/JoshuaFluke/status/1041667768750407680 …
Two incomes and a trust fund
My Uber driver was jazzed he found this great deal until it ended up going for a few hundred thousand above asking. And it’s in the cheap are

### Encode to integers

In [15]:
# get all tweet texts
both_tweets = ''
for i in austen_tweets:
    both_tweets = both_tweets + i.text
for i in elon_tweets:
    both_tweets = both_tweets + i.text

# Convert all tweet texts to numeric
chars = list(set(both_tweets))
char_indices = dict((c, i) for i, c in enumerate(chars))

# Convert austen tweet to numeric set based on all tweets
austen_tweets_num = []
for i, j in enumerate(austen_tweets):
    num_list = [char_indices[char] for char in j.text]
    austen_tweets_num.append(num_list)

# Convert elon tweet to numeric set based on all tweets
elon_tweets_num = []
for i, j in enumerate(elon_tweets):
    num_list = [char_indices[char] for char in j.text]
    elon_tweets_num.append(num_list)

print(len(austen_tweets_num), len(elon_tweets_num))
print("-"*100)
print(austen_tweets_num)
print("-"*100)
print(elon_tweets_num)

181 721
----------------------------------------------------------------------------------------------------
[[105, 69, 85, 91, 36, 98, 69, 85, 91, 36, 98, 69, 85, 91, 36, 98, 69, 58, 91, 64, 43, 126, 41, 1, 69, 58, 126, 82, 31, 69, 1, 64, 98, 2, 82, 69, 67, 98, 91, 67, 85, 98, 59, 67, 126, 84, 59, 82, 58, 126, 82, 82, 98, 64, 59, 84, 91, 45, 97, 53, 7, 79, 32, 45, 35, 47, 85], [59, 59, 59, 41, 91, 82, 69, 98, 36, 98, 41, 69, 84, 85, 91, 62, 98], [112, 31, 2, 82, 69, 126, 53, 69, 104, 91, 33, 125, 36, 98, 69, 83, 98, 98, 41, 69, 82, 64, 104, 126, 41, 1, 69, 53, 91, 64, 69, 31, 33, 41, 76, 64, 98, 76, 62, 69, 91, 53, 69, 104, 98, 2, 64, 62, 69, 2, 41, 76, 69, 126, 82, 125, 62, 69, 41, 98, 36, 98, 64, 69, 84, 91, 45, 98, 69, 98, 36, 98, 41, 69, 64, 98, 45, 91, 82, 98, 85, 104, 69, 84, 85, 91, 62, 98, 69, 82, 91, 69, 58, 91, 64, 43, 126, 41, 1, 69, 83, 33, 82, 69, 45, 126, 85, 85, 126, 91, 41, 62, 69, 91, 53, 69, 67, 98, 91, 67, 85, 98, 69, 31, 2, 36, 98, 69, 76, 126, 98, 76, 69, 2, 62, 6

### Run Model

In [18]:
# Convert to np array for machine learning
X = np.array(austen_tweets_num + elon_tweets_num)

austen_y = np.zeros((len(austen_tweets_num),), dtype=np.int)
elon_y = np.ones((len(elon_tweets_num),), dtype=np.int)
y = np.concatenate((austen_y,elon_y), axis=0)
X.shape, y.shape

((902,), (902,))

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((721,), (181,), (721,), (181,))

In [20]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

max_features = 2000
maxlen = 80
epochs = 10
batch_size = 20

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('x_train shape:', X_train.shape)
print('x_test shape:', X_test.shape)

model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Using TensorFlow backend.


Pad sequences (samples x time)
x_train shape: (721, 80)
x_test shape: (181, 80)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         256000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 387,713
Trainable params: 387,713
Non-trainable params: 0
_________________________________________________________________
None


### Result

In [21]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size)

Instructions for updating:
Use tf.cast instead.
Train on 721 samples, validate on 181 samples
Epoch 1/10
721/721 [==============================] - 21s 29ms/step - loss: 0.5311 - acc: 0.8086 - val_loss: 0.5593 - val_acc: 0.7569
Epoch 2/10
721/721 [==============================] - 14s 20ms/step - loss: 0.4708 - acc: 0.8100 - val_loss: 0.5344 - val_acc: 0.7569
Epoch 3/10
721/721 [==============================] - 13s 18ms/step - loss: 0.4480 - acc: 0.8100 - val_loss: 0.5231 - val_acc: 0.7569
Epoch 4/10
721/721 [==============================] - 13s 18ms/step - loss: 0.4286 - acc: 0.8100 - val_loss: 0.5050 - val_acc: 0.7680
Epoch 5/10
721/721 [==============================] - 14s 19ms/step - loss: 0.4209 - acc: 0.8128 - val_loss: 0.5056 - val_acc: 0.7790
Epoch 6/10
721/721 [==============================] - 14s 20ms/step - loss: 0.4222 - acc: 0.8252 - val_loss: 0.5022 - val_acc: 0.7735
Epoch 7/10
721/721 [==============================] - 16s 22ms/step - loss: 0.4113 - acc: 0.8280 - val

## Part 2 - CNNs
Time to play "find the frog!" Use Keras and ResNet50 to detect which of the following images contain frogs:

In [22]:
!pip install google_images_download

In [23]:
from google_images_download import google_images_download

response = google_images_download.googleimagesdownload()
arguments = {'keywords': "animal pond", "limit": 5, "print_urls": True}
absolute_image_paths = response.download(arguments)



Item no.: 1 --> Item name = animal pond
Evaluating...
Starting Download...
Image URL: https://www.enchantedlearning.com/pgifs/Pondanimals.GIF
Completed Image ====> 1.Pondanimals.GIF
Image URL: https://i.ytimg.com/vi/NCbu0TND9vE/hqdefault.jpg
Completed Image ====> 2.hqdefault.jpg
Image URL: https://pklifescience.com/staticfiles/articles/images/PKLS4116_inline.png
Completed Image ====> 3.PKLS4116_inline.png
Image URL: https://get.pxhere.com/photo/water-animal-pond-wildlife-mammal-fish-eat-fauna-whiskers-vertebrate-otter-mink-marmot-sea-otter-mustelidae-1383482.jpg
Completed Image ====> 4.water-animal-pond-wildlife-mammal-fish-eat-fauna-whiskers-vertebrate-otter-mink-marmot-sea-otter-mustelidae-1383482.jpg
Image URL: http://images.animalpicturesociety.com/images/5d/alligator_animal_on_pond.jpg
Completed Image ====> 5.alligator_animal_on_pond.jpg

Errors: 0



At the time of writing at least a few do, but since the internet changes - it is possible your 5 won't. You can easily verify yourself, and (once you have working code) increase the number of images you pull to be more sure of getting a frog. Your goal is validly run ResNet50 on the input images - don't worry about tuning or improving the model. 

*Hint:* ResNet 50 doesn't just return "frog". The three labels it has for frogs are bullfrog, tree frog, and tailed frog.

Stretch goal - also check for fish.

In [24]:
for i in absolute_image_paths[0]['animal pond']:
    print(i.split("/")[-1:])

['1.Pondanimals.GIF']
['2.hqdefault.jpg']
['3.PKLS4116_inline.png']
['4.water-animal-pond-wildlife-mammal-fish-eat-fauna-whiskers-vertebrate-otter-mink-marmot-sea-otter-mustelidae-1383482.jpg']
['5.alligator_animal_on_pond.jpg']


In [28]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions

def process_img_path(img_path):
  return image.load_img(img_path, target_size=(224, 224))

def img_contains_frog(img):
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    model = ResNet50(weights='imagenet')
    features = model.predict(x)
    results = decode_predictions(features, top=3)[0]
    print(results)
    for entry in results:
        if 'frog' in entry[1]:
            return entry[2], 'Frog is in this picture'
        else:
            return 'Frog is not in this picture'

for i in absolute_image_paths[0]['animal pond']:
    print(i.split("/")[-1:])
    print(img_contains_frog(process_img_path(i)))
    print("-"*50)


['1.Pondanimals.GIF']
[('n03598930', 'jigsaw_puzzle', 0.8680317), ('n06359193', 'web_site', 0.064100206), ('n02834397', 'bib', 0.021264251)]
Frog is not in this picture
--------------------------------------------------
['2.hqdefault.jpg']
[('n01443537', 'goldfish', 0.8495909), ('n01631663', 'eft', 0.067602254), ('n02536864', 'coho', 0.03516372)]
Frog is not in this picture
--------------------------------------------------
['3.PKLS4116_inline.png']
[('n04243546', 'slot', 0.8712449), ('n04476259', 'tray', 0.049936026), ('n03908618', 'pencil_box', 0.023072386)]
Frog is not in this picture
--------------------------------------------------
['4.water-animal-pond-wildlife-mammal-fish-eat-fauna-whiskers-vertebrate-otter-mink-marmot-sea-otter-mustelidae-1383482.jpg']
[('n02442845', 'mink', 0.30976456), ('n02363005', 'beaver', 0.23399007), ('n02361337', 'marmot', 0.20796947)]
Frog is not in this picture
--------------------------------------------------
['5.alligator_animal_on_pond.jpg']
[('n

## Part 3 - AutoML

Use [TPOT](https://epistasislab.github.io/tpot/) to fit a predictive model for the King County housing data, with `price` as the target output variable.

In [1]:
!pip install tpot

In [2]:
import pandas as pd

url = "https://raw.githubusercontent.com/ryanleeallred/datasets/master/kc_house_data.csv"
df = pd.read_csv(url)

df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [3]:
print(df.shape)
df.isnull().sum()

(21613, 21)


id               0
date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64

As with previous questions, your goal is to run TPOT and successfully run and report error at the end. Also, in the interest of time, feel free to choose small `generation=1`and `population_size=10` parameters, so your pipeline runs efficiently. You will want to be able to iterate and test. 

*Hint:* You will have to drop and/or type coerce at least a few variables to get things working. It's fine to err on the side of dropping to get things running - as long as you still get a valid model with reasonable predictive power. 

In [4]:
from tpot import TPOTRegressor
from sklearn.model_selection import train_test_split

X = df.drop(['price','id','date'], axis=1).values
X_train, X_test, y_train, y_test = train_test_split(X, df['price'].values, test_size=0.2)

### Result

In [6]:
tpot = TPOTRegressor(generations=2, population_size=10, verbosity=2)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))

Generation 1 - Current best internal CV score: -23246771549.915924
Generation 2 - Current best internal CV score: -18753456012.815598

Best pipeline: LassoLarsCV(RandomForestRegressor(input_matrix, bootstrap=False, max_features=0.5, min_samples_leaf=18, min_samples_split=7, n_estimators=100), normalize=False)
-23411405727.61556


## Part 4 - More... 

Answer the following questions, with a target audience of a fellow Data Scientist:
* What do you consider your strongest area as a Data Scientist? 
    * My strongest area as a Data Scientist are domain knowledge (business & healthcare) and applying machine learning models to solve real-world problem.
* What area of Data Science would you most like to learn more about and why? 
    * I would probably spend more time learning more about natural language processing and cognitive computing development since their usefulness are applicable to most industries.
* Where do you think Data Science will be in 5 years? 
    * Similar to a boom and bust cycle, I feel that data science is in a booming cycle where its popularity is trending up.  I hope the good time continues for at least 10 years so we can build better and diverse technology as more people getting into the field.

A few sentences per answer is fine. Only elaborate if time allows. Use markdown to format your answers.

Thank you for your hard, and congratulations!! You've learned a lot, and you should proudly call yourself a Data Scientist. 